In [1]:
import pandas as pd
import pymysql
import re,json,os

In [6]:
base = os.environ['BUG_FREE_EUREKA_BASE']
pathTofile = os.path.join(base,'data','exercise06','E-TABM-713_A-MEXP-1551-analytics.tsv')

with open(os.path.join(base,'secrets.json'))as f :
    secrets = json.load(f)

In [16]:
   
# connection to database
conn = pymysql.connect(**secrets['test_db'])
cursor = conn.cursor(pymysql.cursors.DictCursor) # (to get data in form of dict)

# before we get a table, we need to create a database
cursor.execute('create database if not exists biodb')

C:\Users\Dell\Anaconda3\lib\site-packages\pymysql\cursors.py:323: Warning: (1007, "Can't create database 'biodb'; database exists")
  self._do_get_result()


1

In [17]:
#load data into dataframe
df = pd.read_csv(pathTofile, sep = '\t')
df.head()

,Gene ID,Gene Name,Design Element,g1_g2.p-value,g1_g2.t-statistic,g1_g2.log2foldchange
0,MIMAT0000062,MIMAT0000062,A_25_P00010086,0.702506,0.625362,0.078492
1,MIMAT0000062,MIMAT0000062,A_25_P00011584,0.975514,0.065110,0.007410
2,MIMAT0000063,MIMAT0000063,A_25_P00010070,0.139014,2.431636,0.556112
3,MIMAT0000063,MIMAT0000063,A_25_P00010071,0.049728,3.254492,0.811621
4,MIMAT0000064,MIMAT0000064,A_25_P00010072,0.098283,2.697813,0.757066


In [18]:
#create table from data frame

cursor.execute('use biodb')

#fix column name

df.columns = [re.sub('[ .-]','',x) for x in df.columns]
df.to_sql('dea',conn,flavor = 'mysql')

C:\Users\Dell\Anaconda3\lib\site-packages\pandas\core\generic.py:1165: FutureWarning: The 'mysql' flavor with DBAPI connection is deprecated and will be removed in future versions. MySQL will be further supported with SQLAlchemy connectables.
  chunksize=chunksize, dtype=dtype)


ValueError: Table 'dea' already exists.

In [19]:
# check if table is created

cursor.execute("show tables")
cursor.fetchall()

[{'Tables_in_biodb': 'ache'},
 {'Tables_in_biodb': 'dea'},
 {'Tables_in_biodb': 'drugbank'},
 {'Tables_in_biodb': 'new_table_name2'},
 {'Tables_in_biodb': 'uniprot'}]

1. write a query to fetch first 10 lines from dea
2. count number of rows in table
3.calculate average p_value for entire data
4.calculate average p_value  for each gene
5.find the gene with highest fold change , lowest fold change 

In [24]:
#write a query to fetch first 10 lines from dea

cursor.execute('select * from dea LIMIT 10 ')
cursor.fetchall()

[{'DesignElement': 'A_25_P00010086',
  'GeneID': 'MIMAT0000062',
  'GeneName': 'MIMAT0000062',
  'g1_g2log2foldchange': 0.0784915951074403,
  'g1_g2pvalue': 0.70250559386657,
  'g1_g2tstatistic': 0.625362183554288,
  'index': 0},
 {'DesignElement': 'A_25_P00011584',
  'GeneID': 'MIMAT0000062',
  'GeneName': 'MIMAT0000062',
  'g1_g2log2foldchange': 0.00741007045087905,
  'g1_g2pvalue': 0.975513746316429,
  'g1_g2tstatistic': 0.0651101242194695,
  'index': 1},
 {'DesignElement': 'A_25_P00010070',
  'GeneID': 'MIMAT0000063',
  'GeneName': 'MIMAT0000063',
  'g1_g2log2foldchange': 0.556111895299825,
  'g1_g2pvalue': 0.139013519150447,
  'g1_g2tstatistic': 2.43163612513429,
  'index': 2},
 {'DesignElement': 'A_25_P00010071',
  'GeneID': 'MIMAT0000063',
  'GeneName': 'MIMAT0000063',
  'g1_g2log2foldchange': 0.811621134771325,
  'g1_g2pvalue': 0.0497282355402996,
  'g1_g2tstatistic': 3.25449181729957,
  'index': 3},
 {'DesignElement': 'A_25_P00010072',
  'GeneID': 'MIMAT0000064',
  'GeneName':

In [25]:
#count number of rows in table
cursor.execute('select count(*)  as total_row from dea ')
cursor.fetchall()

[{'total_row': 1230}]

In [28]:
#calculate average p_value for entire data
cursor.execute('describe dea')
cursor.fetchall()

[{'Default': None,
  'Extra': '',
  'Field': 'index',
  'Key': 'MUL',
  'Null': 'YES',
  'Type': 'bigint(20)'},
 {'Default': None,
  'Extra': '',
  'Field': 'GeneID',
  'Key': '',
  'Null': 'YES',
  'Type': 'varchar(63)'},
 {'Default': None,
  'Extra': '',
  'Field': 'GeneName',
  'Key': '',
  'Null': 'YES',
  'Type': 'varchar(63)'},
 {'Default': None,
  'Extra': '',
  'Field': 'DesignElement',
  'Key': '',
  'Null': 'YES',
  'Type': 'varchar(63)'},
 {'Default': None,
  'Extra': '',
  'Field': 'g1_g2pvalue',
  'Key': '',
  'Null': 'YES',
  'Type': 'double'},
 {'Default': None,
  'Extra': '',
  'Field': 'g1_g2tstatistic',
  'Key': '',
  'Null': 'YES',
  'Type': 'double'},
 {'Default': None,
  'Extra': '',
  'Field': 'g1_g2log2foldchange',
  'Key': '',
  'Null': 'YES',
  'Type': 'double'}]

In [34]:
# average p_value for the entire data
cursor.execute('Select avg(g1_g2pvalue) Average_pValue_for_table from dea')
cursor.fetchall()

[{'Average_pValue_for_table': 0.4529776486719517}]

In [39]:
# average p_value for each gene
cursor.execute('Select GeneName,avg(g1_g2pvalue)as Avg_pValue from dea GROUP BY g1_g2pvalue LIMIT 10' )
cursor.fetchall()

[{'Avg_pValue': None, 'GeneName': 'MIMAT0000079'},
 {'Avg_pValue': 0.000321778898634328, 'GeneName': 'MIMAT0003227'},
 {'Avg_pValue': 0.00145162156673662, 'GeneName': 'MIMAT0003304'},
 {'Avg_pValue': 0.0022527138479248, 'GeneName': 'MIMAT0000082'},
 {'Avg_pValue': 0.00253015446820666, 'GeneName': 'MIMAT0003227'},
 {'Avg_pValue': 0.00300308814947792, 'GeneName': 'MIMAT0003220'},
 {'Avg_pValue': 0.00355452319136402, 'GeneName': 'MIMAT0000258'},
 {'Avg_pValue': 0.00396649049746904, 'GeneName': 'MIMAT0000082'},
 {'Avg_pValue': 0.00704100264403452, 'GeneName': 'MIMAT0003306'},
 {'Avg_pValue': 0.00744277646273271, 'GeneName': 'MIMAT0003249'}]

In [50]:
# gene with highest and lowest fold change

cursor.execute('Select GeneID,GeneName,max(g1_g2log2foldchange)from dea ')
cursor.fetchall()

# THIS IS WRONG !!!!!!?

[{'GeneID': 'MIMAT0000062',
  'GeneName': 'MIMAT0000062',
  'max(g1_g2log2foldchange)': 2.68544495738703}]

In [48]:
cursor.execute('Select GeneID from dea where g1_g2log2foldchange = (select max(g1_g2log2foldchange) from dea)  ')
cursor.fetchall()

[{'GeneID': 'MIMAT0002177'}]

In [49]:
cursor.execute('Select GeneID from dea where g1_g2log2foldchange = (select min(g1_g2log2foldchange) from dea)  ')
cursor.fetchall()

[{'GeneID': 'MIMAT0000451'}]